<a href="https://colab.research.google.com/github/ChitranshGuha/Cat-Dog-App/blob/master/dogsNcats.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install tensorflow
!pip install kaggle

In [ ]:
import os
from google.colab import files

# Upload Kaggle API key
files.upload()

# Make the Kaggle folder and move the json API key
os.makedirs("/root/.kaggle", exist_ok=True)
os.rename("kaggle.json", "/root/.kaggle/kaggle.json")

# Download the dataset from Kaggle
!kaggle datasets download -d bhavikjikadara/dog-and-cat-classification-dataset

Saving kaggle.json to kaggle.json
Dataset URL: https://www.kaggle.com/datasets/bhavikjikadara/dog-and-cat-classification-dataset
License(s): apache-2.0
100% 772M/775M [00:06<00:00, 142MB/s]
100% 775M/775M [00:06<00:00, 131MB/s]


In [ ]:
import zipfile

# Extract the dataset zip file
with zipfile.ZipFile('dog-and-cat-classification-dataset.zip', 'r') as zip_ref:
    zip_ref.extractall('cats_dogs')

# Check the extracted contents
os.listdir('cats_dogs')


['PetImages']

In [ ]:
import os
print(os.listdir())  # Check the current directory contents


['.config', 'cats_dogs', 'dog-and-cat-classification-dataset.zip', 'sample_data']


In [ ]:
import zipfile

zip_path = "dog-and-cat-classification-dataset.zip"
extract_path = "cats_dogs"

# Extract if not already extracted
if not os.path.exists(f"{extract_path}/train"):
    with zipfile.ZipFile(zip_path, 'r') as zip_ref:
        zip_ref.extractall(extract_path)
    print("Dataset extracted successfully!")
else:
    print("Dataset already extracted.")

In [ ]:
import os
import shutil
from sklearn.model_selection import train_test_split

base_dir = "cats_dogs/PetImages"
train_dir = "cats_dogs/train"
validation_dir = "cats_dogs/validation"

# Create train & validation directories
for folder in ["train", "validation"]:
    for category in ["Dog", "Cat"]:
        os.makedirs(os.path.join("cats_dogs", folder, category), exist_ok=True)

# Move images into train & validation
for category in ["Dog", "Cat"]:
    source = os.path.join(base_dir, category)
    images = os.listdir(source)

    train_images, val_images = train_test_split(images, test_size=0.2, random_state=42)

    for img in train_images:
        shutil.move(os.path.join(source, img), os.path.join(train_dir, category, img))
    for img in val_images:
        shutil.move(os.path.join(source, img), os.path.join(validation_dir, category, img))

print("Dataset split into train/validation successfully!")


Dataset split into train/validation successfully!


In [ ]:
train_dir = "cats_dogs/train"
validation_dir = "cats_dogs/validation"


In [ ]:
import os
from PIL import Image

base_dir = "cats_dogs/PetImages"

for category in ["Cat", "Dog"]:
    folder_path = os.path.join(base_dir, category)
    for filename in os.listdir(folder_path):
        file_path = os.path.join(folder_path, filename)
        try:
            with Image.open(file_path) as img:
                img.verify()  # Check if image is corrupted
        except (IOError, SyntaxError):
            print(f"Corrupt or missing file detected: {file_path}")
            os.remove(file_path)  # Remove corrupted file


In [ ]:
for category in ["Cat", "Dog"]:
    folder_path = os.path.join(base_dir, category)
    for filename in os.listdir(folder_path):
        file_path = os.path.join(folder_path, filename)
        if not filename.lower().endswith(('.jpg', '.jpeg', '.png')):
            new_path = file_path + ".jpg"
            os.rename(file_path, new_path)
            print(f"Renamed: {file_path} → {new_path}")


In [ ]:
train_generator = train_datagen.flow_from_directory(
    train_dir,
    target_size=(IMG_SIZE, IMG_SIZE),
    batch_size=32,
    class_mode='binary',
    subset='training'
)

validation_generator = train_datagen.flow_from_directory(
    validation_dir,
    target_size=(IMG_SIZE, IMG_SIZE),
    batch_size=32,
    class_mode='binary',
    subset='validation'
)


Found 16000 images belonging to 4 classes.
Found 1000 images belonging to 4 classes.


In [ ]:
from tensorflow.keras import layers, models
import tensorflow as tf

# Load pre-trained MobileNetV2 without the top layer
base_model = tf.keras.applications.MobileNetV2(input_shape=(IMG_SIZE, IMG_SIZE, 3),
                                               include_top=False,
                                               weights='imagenet')

# Freeze the base model
base_model.trainable = False

# Add a classification layer on top
model = models.Sequential([
    base_model,
    layers.GlobalAveragePooling2D(),
    layers.Dense(1, activation='sigmoid')
])

# Compile the model
model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

# Train the model
history = model.fit(
    train_generator,
    steps_per_epoch=train_generator.samples // 32,
    epochs=4,
    validation_data=validation_generator,
    validation_steps=validation_generator.samples // 32
)


Epoch 1/4
500/500 ━━━━━━━━━━━━━━━━━━━━ 756s 2s/step - accuracy: 0.9503 - loss: 0.1404 - val_accuracy: 0.9748 - val_loss: 0.0571
Epoch 2/4
500/500 ━━━━━━━━━━━━━━━━━━━━ 764s 1s/step - accuracy: 0.9814 - loss: 0.0504 - val_accuracy: 0.9819 - val_loss: 0.0494
Epoch 3/4
500/500 ━━━━━━━━━━━━━━━━━━━━ 802s 2s/step - accuracy: 0.9851 - loss: 0.0398 - val_accuracy: 0.9819 - val_loss: 0.0499
Epoch 4/4
500/500 ━━━━━━━━━━━━━━━━━━━━ 805s 2s/step - accuracy: 0.9881 - loss: 0.0344 - val_accuracy: 0.9808 - val_loss: 0.0564


In [ ]:
# Save the model as .h5 (Keras format)
model.save('/content/cat_dog_classifier.h5')

In [ ]:
# Convert the model to TensorFlow Lite format
converter = tf.lite.TFLiteConverter.from_keras_model(model)
tflite_model = converter.convert()

# Save the TensorFlow Lite model
tflite_model_path = '/content/cat_dog_classifier.tflite'
with open(tflite_model_path, 'wb') as f:
    f.write(tflite_model)

# You can download the .tflite file from Colab
from google.colab import files
files.download(tflite_model_path)


Saved artifact at '/tmp/tmpzqa_1kay'. The following endpoints are available:

* Endpoint 'serve'
  args_0 (POSITIONAL_ONLY): TensorSpec(shape=(None, 224, 224, 3), dtype=tf.float32, name='keras_tensor_470')
Output Type:
  TensorSpec(shape=(None, 1), dtype=tf.float32, name=None)
Captures:
  136638511559696: TensorSpec(shape=(), dtype=tf.resource, name=None)
  136638350064144: TensorSpec(shape=(), dtype=tf.resource, name=None)
  136638350064528: TensorSpec(shape=(), dtype=tf.resource, name=None)
  136638350064336: TensorSpec(shape=(), dtype=tf.resource, name=None)
  136638350064912: TensorSpec(shape=(), dtype=tf.resource, name=None)
  136638350062992: TensorSpec(shape=(), dtype=tf.resource, name=None)
  136638350063376: TensorSpec(shape=(), dtype=tf.resource, name=None)
  136638350061072: TensorSpec(shape=(), dtype=tf.resource, name=None)
  136638350060880: TensorSpec(shape=(), dtype=tf.resource, name=None)
  136638350063568: TensorSpec(shape=(), dtype=tf.resource, name=None)
  1366383500

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>